## Hybrid(Section毎＋全体チャンク化、re-rank)

本コードはsudachi_user.dicを**空にしてから実行**のこと  
実行後に、戻すことを忘れないこと

```
ノート全体と、Section毎にチャンク化してVector StoreとBM Indexに格納。  
３つの入力領域を＋対応するクエリで検索する＋ノート全体を結合したクエリで検索する。  
各クエリは、VectorSearchとBM25Searchで行い、RRFでランク統合を行い、rerankを行う。
   QueryA -+- embedding - VectorSearch -+-> RRF --> rerank -+- Aggregate -> topK 
           +- Tokenize  - BM25Search   -+                   |
   QueryB -       同上                                     -+
   QueryC -       同上                                     -+
   QueryA+B+C -   同上                                     -+
   
４つのクエリの結果を、同じノート毎に集約する。集約時の計算は相加平均または最大値。  
材料のデータに関しては名寄せを実施。  

pip install rank_bm25
pip install SudachiPy sudachidict_core sudachidict_full
```

### 各種設定

In [ ]:
import re, glob, time
import numpy as np
import pandas as pd
pd.set_option('display.width', 150)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows',  None)
from pprint import pprint
from collections import defaultdict

from sentence_transformers import SentenceTransformer # embedding用
import google.genai as genai
import openai
import cohere
import chromadb
import sudachipy
from sudachipy import Dictionary, SplitMode # 日本語Tokenizer
from rank_bm25 import BM25Okapi, BM25Plus   # BM25
import langchain_core
from langchain_core.documents import Document
from langchain_core.stores    import InMemoryStore # 親チャンク用(未使用)
from sklearn.metrics import ndcg_score

print(f"genai={genai.__version__}, openai={openai.__version__}, " + 
      f"cohere={cohere.__version__}, chromadb={chromadb.__version__}, " +
      f"sudachipy={sudachipy.__version__}, " +
      f"langchain_core={langchain_core.__version__}, ")

genai=1.56.0, openai=2.13.0, cohere=5.20.0, chromadb=1.3.7, sudachipy=0.6.10, langchain_core=1.2.2, 


#### LLMモデル利用定義

In [16]:
# Geminiモデルを指定
GEMINI_EMBEDDING_MODEL = 'gemini-embedding-001'
#GEMINI_LLM_MODEL      = 'gemini-2.0-flash'  # Limit-Rateが高い
#GEMINI_LLM_MODEL      = 'gemini-2.5-flash'  # それなりに賢い
#GEMINI_LLM_MODEL      = 'gemini-2.5-pro'
#GEMINI_LLM_MODEL      = 'gemini-3-pro-preview'

# OpenAIモデルを指定
OPENAI_EMBEDDING_MODEL_SMALL = 'text-embedding-3-small'
OPENAI_EMBEDDING_MODEL_LARGE = 'text-embedding-3-large'

# Cohereモデルを指定
#COHERE_EMBEDDING_MODEL = 'embed-v4.0'
COHERE_LLM_MODEL       = 'command-a-03-2025'
COHERE_RERANK_MODEL     = 'rerank-v3.5'

# Geminiクライアントを作成
with open('keys/Google_API_KEY.txt', 'r') as f:
    api_key = f.read().strip()
GEMINI_CLIENT = genai.Client(api_key=api_key)

# Cohereクライアントを作成
with open('keys/Cohere_API_KEY.txt', 'r') as f:
    api_key = f.read().strip()
COHERE_CLIENT = cohere.ClientV2(api_key=api_key)

# OpenAIクライアントを作成
with open('keys/OpenAI_API_KEY.txt', 'r') as f:
    api_key = f.read().strip()
OPENAI_CLIENT = openai.OpenAI(api_key=api_key)

# Chromaクライアントを作成
CHROMA_CLIENT = chromadb.EphemeralClient()  # インメモリで作成

# PLaMoクライアントを作成
PLAMO_CLIENT = SentenceTransformer('pfnet/plamo-embedding-1b', trust_remote_code=True)

# ruriクライアントを作成
RURI_CLIENT = SentenceTransformer('cl-nagoya/ruri-v3-310m', trust_remote_code=True)

from enum import Enum, auto
class EmbeddingType(Enum): # モデル切り替え用定数の設定
    GEMINI = auto()
    OPENAI_SMALL = auto()
    OPENAI_LARGE = auto()
#    COHERE = auto()  # 未使用
    PLAMO  = auto()
    RURI   = auto()
#======================================
EMBEDDING_MODEL = EmbeddingType.GEMINI   # モデルを指定
#======================================

COHERE(tail): mtWPf58VOAX5Tjhx45Co


No sentence-transformers model found with name pfnet/plamo-embedding-1b. Creating a new one with mean pooling.


#### embedding関数

In [3]:
def get_embeddings(texts):
    """共通のEmbedding関数"""

    # 単一文字列でも対応
    single_input = False
    if isinstance(texts, str):
        texts = [texts]
        single_input = True

    match EMBEDDING_MODEL:

        # Gemini
        case EmbeddingType.GEMINI:
            # Embedding取得
            response = GEMINI_CLIENT.models.embed_content(
                model=GEMINI_EMBEDDING_MODEL,
                contents=texts
            )
            embeddings = [e.values for e in response.embeddings]

        # OpenAI
        case EmbeddingType.OPENAI_SMALL | EmbeddingType.OPENAI_LARGE:
            if EMBEDDING_MODEL == EmbeddingType.OPENAI_LARGE:
                OPENAI_EMBEDDING_MODEL = OPENAI_EMBEDDING_MODEL_LARGE 
            else:
                OPENAI_EMBEDDING_MODEL = OPENAI_EMBEDDING_MODEL_SMALL 
            # Embeddings を取得
            response = OPENAI_CLIENT.embeddings.create(
                model=OPENAI_EMBEDDING_MODEL,
                input=texts
            )
            # レスポンスからベクトル抽出（resp.data は入力ごとの要素を持つ）
            embeddings = [d.embedding for d in response.data]

        # PLaMo
        case EmbeddingType.PLAMO:
            embeddings = PLAMO_CLIENT.encode(
                texts,
                batch_size=16,
                show_progress_bar=False,
                convert_to_numpy=True
            )

        # Ruri
        case EmbeddingType.RURI:
            embeddings = RURI_CLIENT.encode(
                texts, 
                batch_size=16, 
                convert_to_numpy=True, 
                show_progress_bar=False
            )

    # 単一入力だったら一次元のリストを返す
    if single_input:
        return embeddings[0]

    return embeddings

# 確認
print(f"EMBEDDING_MODEL={EMBEDDING_MODEL}")
for texts in ['今日はいい天気です。', ['今日はいい天気です。','明日はどうでしょうか？']]:
    embedded_text = get_embeddings(texts)
    if isinstance(embedded_text, np.ndarray):
        print(f"{type(embedded_text)}, {embedded_text.shape}, {embedded_text.flatten()[:4]} ...")
    else:
        try:
            print(f"{type(embedded_text)}, {len(embedded_text)}, {embedded_text[0][:4]} ...")
        except:
            print(f"{type(embedded_text)}, {len(embedded_text)}, {embedded_text[:4]} ...")

EMBEDDING_MODEL=EmbeddingType.GEMINI
<class 'list'>, 3072, [-0.019278863, -0.0022764413, 0.0021871584, -0.08121658] ...
<class 'list'>, 2, [-0.019278863, -0.0022764413, 0.0021871584, -0.08121658] ...


#### 日本語 Tokenizer

入力文を、形態素解析エンジンでトークン化、必要な品詞のみのトークンのリストを返す  

In [4]:
### すだち
dict_obj = Dictionary(dict='full', config_path='./sudachi.json')
tokenizer = dict_obj.create()

# <Morpheme(撹拌条件, 0:4, (1, 0))>, 入力, 辞書マッチ範囲, (1:user.dic, 単語ID)
print([dict_obj.lookup('撹拌条件')]) # User_dic確認 辞書になければ[]
print([dict_obj.lookup('hba1c')])

def tokenize(text, splitmode='C', pos=False):

    match splitmode.upper():
        case 'A':
            splitmode = SplitMode.A # 超細かい
        case 'B':
            splitmode = SplitMode.B # 細かい
        case _:
            splitmode = SplitMode.C

    tokens = tokenizer.tokenize(text, splitmode) # Defult = SplitMode.C
    if pos:
        pprint([(t.surface(), t.part_of_speech()) for t in tokens])
    tokens = [t.surface() for t in tokens]
    return tokens

def tokenize_and_filter_by_pos(
    text, 
    target_pos=['名詞','動詞','形容詞','副詞','接頭詞'],
    pos=False
):
    # 入力文から指定された品詞のトークンを抽出
    tokens = list(tokenizer.tokenize(text))
    if pos:
        pprint([(t.surface(), t.part_of_speech()) for t in tokens])
    parts_of_speech = [t.part_of_speech() for t in tokens]                             # 品詞情報を抽出
    is_target_pos = [bool(set(p) & set(target_pos)) and 
                     not (bool(set(p) & set(['動詞'])) and
                          bool(set(p) & set(['非自立', '非自立可能'])))
                                                             for p in parts_of_speech] # 必要品詞で抽出
    #pprint(is_target_pos)
    extracted_tokens = [t.surface() for (t, f) in zip(tokens, is_target_pos) if f]
    return extracted_tokens

def tokenize_detail_info(text):
    tokens = tokenizer.tokenize(text, SplitMode.C)
    print(f"# INPUT: {[m.surface() for m in tokens]}")
    for m in tokens:
        print(f"\nsurface(): {m.surface()} {'-'*10}",
              f"\ndictionary_id(0:System, 1:User): {m.dictionary_id()}", 
              f"\ndictionary_form(見出し): {m.dictionary_form()}", 
              f"\nnormalized_form(正規化表記): {m.normalized_form()}",
              f"\npart_of_speech(品詞): {m.part_of_speech()}",
              f"\npart_of_speech_id(): {m.part_of_speech_id()}",
              f"\nraw_surface(入力): {m.raw_surface()}",
              f"\nreading_form(読み): {m.reading_form()}",
              f"\nsynonym_group_ids(同義語辞書でのID): {m.synonym_group_ids()}",
              f"\nword_id(): {m.word_id()}",
              f"\nis_oov(未知語): {m.is_oov()}",
             )

pprint(tokenize('① 今日は、良い天気です。', pos=True))
pprint(tokenize('p-トルエンスルホン酸', pos=True))
pprint(tokenize_and_filter_by_pos('① 今日は、良い天気です。', pos=True))
pprint(tokenize_detail_info('実験装置'))

[<MorphemeList[
]>]
[<MorphemeList[
]>]
[('①', ('名詞', '数詞', '*', '*', '*', '*')),
 (' ', ('空白', '*', '*', '*', '*', '*')),
 ('今日', ('名詞', '普通名詞', '副詞可能', '*', '*', '*')),
 ('は', ('助詞', '係助詞', '*', '*', '*', '*')),
 ('、', ('補助記号', '読点', '*', '*', '*', '*')),
 ('良い', ('形容詞', '非自立可能', '*', '*', '形容詞', '連体形-一般')),
 ('天気', ('名詞', '普通名詞', '一般', '*', '*', '*')),
 ('です', ('助動詞', '*', '*', '*', '助動詞-デス', '終止形-一般')),
 ('。', ('補助記号', '句点', '*', '*', '*', '*'))]
['①', ' ', '今日', 'は', '、', '良い', '天気', 'です', '。']
[('①', ('名詞', '数詞', '*', '*', '*', '*')),
 (' ', ('空白', '*', '*', '*', '*', '*')),
 ('今日', ('名詞', '普通名詞', '副詞可能', '*', '*', '*')),
 ('は', ('助詞', '係助詞', '*', '*', '*', '*')),
 ('、', ('補助記号', '読点', '*', '*', '*', '*')),
 ('良い', ('形容詞', '非自立可能', '*', '*', '形容詞', '連体形-一般')),
 ('天気', ('名詞', '普通名詞', '一般', '*', '*', '*')),
 ('です', ('助動詞', '*', '*', '*', '助動詞-デス', '終止形-一般')),
 ('。', ('補助記号', '句点', '*', '*', '*', '*'))]
['①', '今日', '良い', '天気']


#### 材料データ名寄せ関数

In [5]:
# 名寄せデータ
Entity_Resolution_dic = {
    r"f(ab')2": r"f(ab')2コートラテックス",
    r"IgG":     r"IgGコートラテックス",
    r"純水":    r"精製水",
}

#分解パターンの定義
# 正規表現
# 末尾の数量、単位の分離 (省略可)
qty_unit_pattern = re.compile(
    r"""
    \s*[:：]*\s*             # 0個以上の空白、セパレータ、空白
    (?P<qty>[\d.,]+)         # 数量 (数値)
    \s*                      # 0個以上の空白
    (?P<unit>[kKmnμµ]{0,1}[a-zA-Z]+|[日時分秒℃度%]|°[CK]|rpm|mol|min|sec)?  # 単位 (省略可) <= 工夫の余地あり
    $                        # 末尾
    """,
    re.VERBOSE
)

ZEN2HAN = str.maketrans(
    "".join(chr(0xff01 + i) for i in range(94))+"　", # Zen
    "".join(chr(0x21   + i) for i in range(94))+" "   # Han
)

def parse_material(line, trace=False):
    '''
    材料行を分解
    材料と数量単位は、空白またはセパレータ[:：]で分かち書きされている
    以下に分解
    1. 先頭[-*・] (無い場合がある) => 無視
    2. 項番: 例: ①,... (1),...   (無い場合がある) => 無視
    3. 材料: 材料名, 補足
      3.1 材料名 数値を含む場合あり => 変数 mate に格納
      3.2 補足「()（）」で囲まれた補足情報が続く場合もあり => ()含め変数 add に格納
    4. 数量, 単位: 無い場合もあり、 数量と単位に間に空白がある場合と無い場合がある
    　4.1 数量(数値、半角または全角、「,.」を含む) => 変数 qtyに格納　
      4.2 単位((L,g)x(そのまま、k,m,μ))、ignore case) => 変数 unit に格納
    '''
    line = line.strip().replace("　", " ")                 # 前後余白を削除、全角スペースを半角に
    line = re.sub(r'^[\-\*]+\s*', '', line)                # 先頭の "-", "*", "•" を除去
    line = re.sub(r'^\s*(\(?\d+\)?|[①-⑳])\s*', '', line)   # 項番（①, ②, (1), 1. など）を除去
    line = line.translate(ZEN2HAN).strip()                 # 数字記号は半角に
    if trace:
        print(f"## {line}")

    # 1. 正規化：セパレータは「空白 or : or ：」
    #    最後の数量部分を優先的に捉えるため "数量+単位" をパターン化して先に抽出
    qty = ''; unit = ''
    m = qty_unit_pattern.search(line)
    if m:
        qty  = m.group('qty')
        unit = m.group('unit')
        # 数量部分を取り除いて「材料側」だけにする
        line_material = line[:m.start()].rstrip()
    else:
        # 数量・単位なし
        line_material = line.strip()

    # 2. 材料名末尾に続く補足括弧を抽出（複数対応）
    #   材料名内部にある括弧は mate に含める
    add = ''
    add_list = []

    # 補足括弧は、文末方向に連続しているものだけ抽出
    add_pattern = re.compile(r"(\([^)]*\)|（[^）]*）)$")

    tmp = line_material
    while True:
        m = add_pattern.search(tmp)
        if not m:
            break
        add_list.insert(0, m.group(1))  # 先頭に挿入して順序維持
        tmp = tmp[:m.start()].rstrip()

    if add_list:
        add = ''.join(add_list)
        mate = tmp.strip()
    else:
        mate = line_material.strip()

    mate_dic = {'material': mate, 'add': add, 'qty': qty, 'unit': unit}
    if trace:
        print(f"## ## {mate_dic}")
    mate_dic = {k: v for k, v in mate_dic.items() if v != '' and v is not None} # 値が''の要素をdrop
    if trace:
        print(f"## ## {mate_dic}")

    return mate_dic


def parse_and_normalize_materials(material_section, trace=False):
    materials = ''
    lines = material_section.strip().split('\n')
    if trace:
        change = []
    for line in lines:
        line = line.strip()
        if line =='':
            continue
        mate_dic = parse_material(line, trace=trace) # 「材料名 (補足) 数量 単位」に分解
        if mate := mate_dic.get('material', False):  # 材料名があれば、
            normalized_mate = Entity_Resolution_dic.get(mate, mate) # 名寄せ
            mate_dic['material'] =  normalized_mate                 # 名寄せ
            if trace:
                change.append('->') if normalized_mate != mate else change.append('--')            
        materials += ' '.join(list(mate_dic.values())) + '\n'

    if trace:
        for i, (org, chg, nor) in enumerate(zip(lines, change, materials.split('\n')),start=1):
            print(f"{i}, {org} {chg} {nor}")
    return materials

# 動作確認
tests = '''
- ① 純水：20 μL
* (2) エタノール: 5,000mol
精製水 100°K
水:　１０ ml
水  10秒
食塩水(5%) 10ml
- ⑥ IgG（粒径 100 nm）：1 mL
- ⑦ f(ab')2（粒径 200 nm）：1 mL
- ⑥ IgGコートラテックス（粒径 100 nm）：1 mL
'''
pprint(parse_and_normalize_materials(tests, trace=False))

('精製水 20 μL\n'
 'エタノール 5,000 mol\n'
 '精製水 100 °K\n'
 '水 10 ml\n'
 '水 10 秒\n'
 '食塩水 (5%) 10 ml\n'
 'IgGコートラテックス (粒径 100 nm) 1 mL\n'
 "f(ab')2コートラテックス (粒径 200 nm) 1 mL\n"
 'IgGコートラテックス (粒径 100 nm) 1 mL\n')


#### データ読込関数
データを、ノート名・セクションの辞書の形で返す

In [6]:
def read_dataset(doc_dir='notes', trace=False): # -> dic
    """実験ノート(md)の読み込み"""
    indata_section_names = ['目的','材料','実験手順'] # 入力データでのSection名

    # doc_dir以下のすべての .md ファイルを取得
    md_files = glob.glob(f"./{doc_dir}/*.md")

    # レシピを辞書に格納
    notes_dic = {}
    for i, file_path in enumerate(md_files, start=1):
        with open(file_path, "r", encoding="utf-8") as f:
            note = f.read().strip()
            name = re.search(r"^# (.+)$", note, re.MULTILINE).group(1).strip()
            notes_dic[name] = note.strip()
    print(f"\ndoc_dir={doc_dir}, 読みこんだレシピの数: {len(notes_dic)}")
    if trace:
        first_key = list(notes_dic.keys())[0]; print(notes_dic[first_key]) # 最初のレシピ

    # セクションごとに辞書化
    section_pattern = re.compile(r'^## (.*?)\n(.*?)(?=\n## |\Z)', re.MULTILINE | re.DOTALL)
    for i, (name, note) in enumerate(notes_dic.items()):
        if trace:
            print(f"## {name} ====")
        matches = section_pattern.findall(note)  # セクションごとに分離辞書化
        result_dic = {title.strip(): content.strip() for title, content in matches}

        # Section名を標準化
        for (std_name, indata_name) in zip(SECTION_NAMES, indata_section_names):
            if std_name != indata_name:
                result_dic[std_name] = result_dic.pop(indata_name)
        if MATE_NORMALIZE:
            result_dic['材料'] = parse_and_normalize_materials(result_dic['材料'], trace=trace) # 材料の正規化

        # allの追加
        result_dic['all'] = '\n\n'.join([f"## {s}\n\n{result_dic[s]}" for s in SECTION_NAMES])

        notes_dic[name] = result_dic
    first_key = list(notes_dic.keys())[0]
    print(f"\n辞書化したノートの数: {len(notes_dic)}\nFirst: {first_key}: {notes_dic[first_key].keys()}")
    pprint(notes_dic[first_key])

    return notes_dic

#### Vector Store/BM25 Index 作成関数
実験ノートデータを読み込み、Section毎＋全体 毎に、Vector Store および MB25 Indexを作成  
Vector Store では Metadataに、(1)ノート名、(2)セクション名or'all'を付与

In [7]:
from tqdm import tqdm
def setup_stores(notes_dic):
    """
    コレクション,インデックスのセットアップ
    Arg:
        notes_dic: {'all': [ノート1,ノート2,...], '目的':[ノート1,ノート1,...], ...}
    return:
        collection:    VectorStore
        bm25index_dic: {'all', BM25index_all, '目的', tokenized_purposes, ...}
        doc_ids:       ['doc'ノート1名'_all', ...]
    """

    # collection作成 (すでに在ったら削除)
    collection_name = 'notes'
    try:
        CHROMA_CLIENT.delete_collection(collection_name)
    except:
        pass
    collection = CHROMA_CLIENT.create_collection(
        name = collection_name,
        metadata={'hnsw:space': 'cosine'}  # 距離メトリック = 'cosine'
    )

    doc_ids  = []
    tokenized_alls      = []
    tokenized_purposes  = []
    tokenized_materials = []
    tokenized_methods   = []
    names = list(notes_dic.keys())    # ノート名
    n = 10 # 一度に処理するノート数（x section数 = バッチサイズ） token size limit対策
    for i in tqdm(range(0, len(names), n), mininterval=2, ncols=80): # バッチ毎に
        page_contents = []
        metadatas     = []
        ids           = []

        for j in range(i, min(i+n, len(names))): # ノート毎に
            name = names[j]
            note_dic = notes_dic[name]
            doc_ids.append(name)

            for k, s in enumerate(note_dic.keys(), start=1): # section毎に
                doc_id = f"{name}_{s}"
                ids.append(doc_id)    # id: ユニークな文字列
                
                # for Vector store ==========================
                section = f"## {s}:\n{note_dic[s].strip()}"
                page_contents.append(section)
                metadatas.append({'source': name, 'section': s})

                # for BM25 Index ==========================
                tokenized_section = tokenize_and_filter_by_pos(section)
                match s:
                    case s if s=='all':
                        tokenized_alls.append(tokenized_section)
                    case s if s==SECTION_NAMES[0]:
                        tokenized_purposes.append(tokenized_section)
                    case s if  s==SECTION_NAMES[1]:
                        tokenized_materials.append(tokenized_section)
                    case s if  s==SECTION_NAMES[2]:
                        tokenized_methods.append(tokenized_section)
                
                #if (name in ['実験ノート ID1-2','実験ノート ID2-16']) and s=='all':
                #    print(f"## {name} section={s}\n{section}")

        # for Vector store ==========================

        # Embeddingの取得
        doc_embs = get_embeddings(page_contents)

        # ChromaDBへ一括で追加
        collection.add(
            ids=ids,
            embeddings=doc_embs,
            documents=page_contents,
            metadatas=metadatas
        )
        
    # for BM25 Index ==========================
    bm25index_dic = {
        'all': BM25Plus(tokenized_alls),
        SECTION_NAMES[0]: BM25Plus(tokenized_purposes),
        SECTION_NAMES[1]: BM25Plus(tokenized_materials),
        SECTION_NAMES[2]: BM25Plus(tokenized_methods),
    }

    print(f"\n## Vector Store、BM25 Indexに追加された文書数: {collection.count()}")

    return collection, bm25index_dic, doc_ids

### データ読込・Stores作成実行

In [8]:
# クライアントと読み込み、コレクションを初期化
SECTION_NAMES = ['目的','材料','手順'] # 正規化時に付与するセクション名
doc_dir = '実験ノート1206'
MATE_NORMALIZE = False # 材料名寄せフラグ
print(f"## EMBEDDING_MODEL={EMBEDDING_MODEL}, MATERIAL_NORMALIZE={MATE_NORMALIZE}")

notes_dic = read_dataset(doc_dir, trace=False)   # データの読み込み

collection, bm25index_dic, doc_ids = setup_stores(notes_dic) # Vector/MB25 Storeの作成

## EMBEDDING_MODEL=EmbeddingType.GEMINI, MATERIAL_NORMALIZE=True

doc_dir=実験ノート1206, 読みこんだレシピの数: 340

辞書化したノートの数: 340
First: 実験ノート ID1-1: dict_keys(['目的', '材料', '手順', 'all'])
{'all': '## 目的\n'
        '\n'
        '現行HbA1c測定試薬（HbA1c捕捉抗体1（マウスモノクローナル，全長IgG）とHbA1c検出抗体1（マウスモノクローナル，HRP標識，全長IgG）、NaOH '
        '10mLとTris '
        '10mL、ラテックス粒径100nmと200nm）の条件を基準条件として設定し、感度・直線性・バックグラウンドなど基本性能を初期評価する。\n'
        '\n'
        '## 材料\n'
        '\n'
        '精製水 20 mL\n'
        'HbA1c捕捉抗体1 (マウスモノクローナル,全長IgG) 1 mL\n'
        'HbA1c検出抗体1 (マウスモノクローナル,HRP標識,全長IgG) 1 mL\n'
        'NaOH 10 mL\n'
        'Tris 10 mL\n'
        'IgGコートラテックス (粒径 100 nm) 1 mL\n'
        "f(ab')2コートラテックス (粒径 200 nm) 1.0 mL\n"
        '\n'
        '\n'
        '## 手順\n'
        '\n'
        '### 手順1\n'
        '\n'
        '1．試薬Aを作成\n'
        '1-1.①～③の分量を測定＆記録して、ビーカーに入れる\n'
        '1-2.スタラーで撹拌する\n'
        '\u3000撹拌条件\n'
        '\u3000\u3000回転数：80±20rpm、撹拌時間：5±2分、撹拌温度：22±3℃\n'
        '1-3.④～⑦の分量を測定＆記録して、ビーカーに入れる\n'
  

100%|███████████████████████████████████████████| 34/34 [01:08<00:00,  2.02s/it]


## Vector Store、BM25 Indexに追加された文書数: 1360


### Hybrid(Vector, Keyword)検索
1. クエリの辞書データを入力。  
    {Section名 or 'all' : クエリ, ...}  
2. 辞書データのkey値に対応するmetadataに基づき、Hybrid検索クエリを発行。  
   ・Vector検索とKeyword検索(BM25) を実行し(TopK=8*Kとする)、両順位情報を元にRRFで順位統合(TopK=6*K)する。  
   ・各Hybrid検索の結果を、リランクする(TopK=4*K)。  
3. リランクの結果を元に、ノート名単位に集計(相加平均or最大値)し、最終順位スコアとする。  

In [9]:
### BM25
bm25index_alls      = bm25index_dic['all']
bm25index_purposes  = bm25index_dic[SECTION_NAMES[0]]
bm25index_materials = bm25index_dic[SECTION_NAMES[1]]
bm25index_methods   = bm25index_dic[SECTION_NAMES[2]]
doc_names = list(dict.fromkeys([d.split('_', 1)[0] for d in doc_ids]))

text1 = '''
### 手順1

1．試薬Aを作成
1-1.①～③の分量を測定＆記録して、ビーカーに入れる
1-2.スタラーで撹拌する
　撹拌条件
　　回転数：80±20rpm、撹拌時間：5±2分、撹拌温度：22±3℃
1-3.③～⑥の分量を測定＆記録して、ビーカーに入れる
1-4.スタラーで撹拌する
　撹拌条件
　　回転数：120±20rpm、撹拌時間：10±2分、撹拌温度：22±3℃
1-5.ビーカーを超音波分散機に設置して、超音波分散を実施
　超音波分散条件
　　100秒×1回、Amp：50%
'''

tokenized_query = tokenize_and_filter_by_pos(text1)        # トークン化
print(tokenized_query)

## 検索 ====================
# トークン列でインデックスを検索（スコアリング）
# スコアはコーパス内の各文書に対する関連度（BM25スコア）です
bm25scores = bm25index_methods.get_scores(tokenized_query) # methodsをtext1で検索
print(len(bm25scores), len(doc_names))

bm25score_df = pd.DataFrame({'doc_names': doc_names, 'bm25score':bm25scores})
print(f"bm25indexをtext1で検索")
display(bm25score_df.sort_values('bm25score', ascending=False).head(20))

['手順', '1', '1', '試薬', 'A', '作成', '1', '1.①', '③', '分量', '測定', '記録', 'ビーカー', '入れる', '1', '2', 'スタラー', '撹拌', '撹拌', '条件', '回転数', '80', '20', 'rpm', '撹拌', '時間', '5', '2', '分', '撹拌', '温度', '22', '3', '℃', '1', '3.③', '⑥', '分量', '測定', '記録', 'ビーカー', '入れる', '1', '4', 'スタラー', '撹拌', '撹拌', '条件', '回転数', '120', '20', 'rpm', '撹拌', '時間', '10', '2', '分', '撹拌', '温度', '22', '3', '℃', '1', '5', 'ビーカー', '超音波', '分散', '機', '設置', '超音波', '分散', '実施', '超音波', '分散', '条件', '100', '秒', '1', '回', 'Amp', '50', '%']
340 340
bm25indexをtext1で検索


,doc_names,bm25score
99,実験ノート ID1-99,36.325942
98,実験ノート ID1-98,36.325942
97,実験ノート ID1-97,36.325942
96,実験ノート ID1-96,36.325942
31,実験ノート ID1-37,36.325942
30,実験ノート ID1-36,36.325942
29,実験ノート ID1-35,36.325942
28,実験ノート ID1-34,36.325942
27,実験ノート ID1-33,36.325942
26,実験ノート ID1-32,36.325942


#### RRF関数

RRF (Reciprocal Rank Fusion)  
順位同士の順位データを統合し、総合スコア(順位)を算出  
上位の順位ほど大きなボーナスポイントが付く  
各順位の元となったスコアは考慮しない     

In [10]:
# RRFの平滑化定数
RRF_K = 60 

def weighted_reciprocal_rank_fusion(
    rankings: list[list[str]],  # 複数のランキングIDリスト (例: [bm25_results, vector_results])
    weights:  list[float]       # 各ランキングの重み (例: [60, 40])
) -> list[tuple[str, float]]:
    """
    二組のランキング情報とその重みを受け取り、RRFで統合スコアを計算する関数 (Weighted RRF)。
    
    Args:
        rankings: 各リトリーバーから得られた文書IDのランキングリストのリスト。
                  [['ID3','ID2','ID1'], ['ID3','ID2','ID1']]
        weights: 各ランキングに対応する重みのリスト。
                  [0.6, 0.4]        
    Returns:
        文書IDと最終スコアのタプルを要素とする、スコア降順のリスト。
    """
    
    if len(rankings) != len(weights):
        raise ValueError("ランキングの数と重みの数が一致しません。")
        
    fused_scores: Dict[str, float] = defaultdict(float)
    
    # 1. 各リトリーバーのランキングをループし、スコアを計算
    for i, rank_list in enumerate(rankings):
        weight = weights[i]
        
        # リスト内の各文書とその順位をループ
        for rank, doc_id in enumerate(rank_list):
            # 順位は0から始まるため、r_i は rank + 1
            r_i = rank + 1 
            
            # Weighted RRFスコアを計算し、合計する
            # S_d = w_i * [ 1 / (r_i + k) ]
            score_contribution = weight * (1 / (r_i + RRF_K))
            fused_scores[doc_id] += score_contribution

    # 2. スコアが高い順にソート
    # 結果をタプル (文書ID, スコア) のリストとして返す
    sorted_ranking = sorted(
        fused_scores.items(), key=lambda item: item[1], reverse=True
    )
    return sorted_ranking

# ----------------------------------------------------
# テスト

# 検索結果の定義 (文書IDはユニークな文字列)
test_df = pd.DataFrame({
    'source'  : ['Doc_A', 'Doc_B', 'Doc_C', 'Doc_D', 'Doc_E', 'Doc_F'], 
    'vector_dist': [2, 1, 3, 6, np.nan, 9], # ベクトル検索の結果(distination)
    'bm25_score':  [8, 3, 5, 2, 1, np.nan]  # キーワード検索の結果(Score)
})
bm25_results   = test_df.dropna(subset=['bm25_score'] ).sort_values('bm25_score',  ascending=False)['source'].dropna()
vector_results = test_df.dropna(subset=['vector_dist']).sort_values('vector_dist', ascending=True )['source'].dropna()
# 重み: 前者(BM25/キーワード一致) を重視する設定 (合計1.0になるように設定)
weights = [60, 40]

print(f"### テスト")
display(test_df)
print(f"weights={weights}\nvector_results:\n{vector_results}, \nbm25_results:\n{bm25_results}")

# RRFの実行 ------------------------------------------------------------
all_rankings = [vector_results, bm25_results]
hybrid_ranking = weighted_reciprocal_rank_fusion(all_rankings, weights)
# ---------------------------------------------------------------------

pprint(hybrid_ranking)
rrf_result_df = pd.DataFrame(hybrid_ranking, columns=['source', 'rrf_score'])
test_df =  pd.merge(test_df, rrf_result_df, on='source', how='left')
display(test_df.sort_values('rrf_score', ascending=False))

,source,vector_dist,bm25_score
0,Doc_A,2.0,8.0
1,Doc_B,1.0,3.0
2,Doc_C,3.0,5.0
3,Doc_D,6.0,2.0
4,Doc_E,NaN,1.0
5,Doc_F,9.0,NaN


weights=[60, 40]
vector_results:
1    Doc_B
0    Doc_A
2    Doc_C
3    Doc_D
5    Doc_F
Name: source, dtype: object, 
bm25_results:
0    Doc_A
2    Doc_C
1    Doc_B
3    Doc_D
4    Doc_E
Name: source, dtype: object
[('Doc_A', 1.6234796404019038),
 ('Doc_B', 1.6185271922976843),
 ('Doc_C', 1.597542242703533),
 ('Doc_D', 1.5625),
 ('Doc_F', 0.9230769230769231),
 ('Doc_E', 0.6153846153846154)]


,source,vector_dist,bm25_score,rrf_score
0,Doc_A,2.0,8.0,1.623480
1,Doc_B,1.0,3.0,1.618527
2,Doc_C,3.0,5.0,1.597542
3,Doc_D,6.0,2.0,1.562500
5,Doc_F,9.0,NaN,0.923077
4,Doc_E,NaN,1.0,0.615385


#### Hybrid検索関数

In [11]:
### Hybrid検索 =============

# Retrive関数 -----------------
def hybrid_retrieve(
    queries, collection, bm25index_dic, doc_ids, 
    k=10, hybrid_weight=[50, 50], aggr_strategy='MEAN', trace=False
): 
    # queries: [目的クエリ,材料クエリ,手順クエリ]
    # collection: vector search用コレクション
    # bm25index_dic: 4個の BM25 Search用 Index の辞書
    # doc_ids: ノート名称のStoreへの格納順リスト
    # hybrid_weight: RRFでのVector/BM25検索のウェイト、デフォルト[50, 50]
    # aggr_strategy: ノート名で集約時にスコアの算出戦略 'mean' or 'max'
    #          -> [[ノート名, 順位ランク], ...] k個のリストのリスト

    bm25index_alls      = bm25index_dic['all']
    bm25index_purposes  = bm25index_dic[SECTION_NAMES[0]]
    bm25index_materials = bm25index_dic[SECTION_NAMES[1]]
    bm25index_methods   = bm25index_dic[SECTION_NAMES[2]]
    aggr_strategy = aggr_strategy.upper()

    if trace:
        print(f"### Hybrid Retrieve: k={k}, hybrid_weight={hybrid_weight}," +\
              f"\nquery=\n「{queries}」") 

    # 材料を正規化 =====================================
    if MATE_NORMALIZE:
        queries[1] = parse_and_normalize_materials(queries[1])
        if trace:
            print(queries[1])

    # allクエリを追加
    queries = ['\n'.join([f"## {s}\n{q.strip()}" for (q, s) in zip(queries, SECTION_NAMES) if q])] + queries
    queries_dic = {s: q if q != '' else '' for q, s in zip(queries, ['all'] + SECTION_NAMES)}
    
    # rerank結果を集約するDFを作成
    conc_reranked_df = pd.DataFrame({
        'doc':    pd.Series(dtype=str),   # 検索された内容
        'source': pd.Series(dtype=str),   # ノート名
        'score':  pd.Series(dtype=float)  # Retricvaでの距離計算結果
    })

    n_queries = 0
    for i, section in enumerate(['all'] + SECTION_NAMES): # セクション毎Retrive
        query = queries_dic[section].strip()
        if query == '':
            continue

        if trace:
            print(f"### Retrieve: {section}: query =\n「{query}」, k={k} ") 

        n_queries += 1
        # --- クエリをembedding ---
        query_emb = get_embeddings(query)
    
        # ChromaDBで類似検索（＝retrieval）=======
        results = collection.query(
            query_embeddings=query_emb,
            n_results = 6*k,
            include = ['documents', 'metadatas', 'distances'],
            where = {'section': section}   # 必要なSectionのみを検索(metadataでのfilter条件)
        )
        retrieved_docs    = results['documents'][0]
        retrieved_sources = [m['source'] for m in results['metadatas'][0]]
        retrieved_dists   = results['distances'][0]
        retrieved_df = pd.DataFrame({
            'doc':    retrieved_docs,
            'source': retrieved_sources,
            'dist':   retrieved_dists
        })
        retrieved_df['section'] = section
        retrieved_df = retrieved_df.sort_values('dist', ascending=True).head(8*k)

        # BM25検索 =============
        tokenized_query = tokenize_and_filter_by_pos(query) # トークン化
        match section:                      # Section毎のBM25Indexを選択
            case s if s=='all':
                bm25scores = bm25index_alls.get_scores(tokenized_query)     # allを検索
            case s if s==SECTION_NAMES[0]:
                bm25scores = bm25index_purposes.get_scores(tokenized_query) # purposを検索
            case s if  s==SECTION_NAMES[1]:
                bm25scores = bm25index_materials.get_scores(tokenized_query) # materialを検索
            case s if  s==SECTION_NAMES[2]:
                bm25scores = bm25index_methods.get_scores(tokenized_query)   # methodを検索

        # 結果をDF化(計算しやすいように)
        bm25score_df = pd.DataFrame({'source': doc_ids, 'bm25score':bm25scores})
        bm25score_df = bm25score_df.sort_values('bm25score', ascending=False).head(6*k)

        # RRF: Vector & BM25 Search rank fusion (ランク統合) =============
        rff_ranking = weighted_reciprocal_rank_fusion(
            [retrieved_df['source'], bm25score_df['source']], hybrid_weight
        )
        # RRFの結果に、doc(検索されたデータ本文)、各検索のスコアをマージ
        rrf_result_df = pd.DataFrame(rff_ranking, columns=['source', 'rrf_score'])
        rrf_result_df = pd.merge(rrf_result_df , retrieved_df, on='source', how='left')
        rrf_result_df = pd.merge(rrf_result_df , bm25score_df, on='source', how='left')
        rrf_result_df = rrf_result_df[['doc','source','rrf_score','dist','bm25score','section']]

        # doc (データ本文)の抜けを補完 (BM25Indexはid名やdocの情報を持たず、返さないので)
        # rerankするには、doc情報が必要
        doc_nan_ids = (rrf_result_df.query("doc.isnull()")['source']+'_'+section).values.tolist()
        if doc_nan_ids != []:
            # Collection内のdoc情報を取得DF化
            nan_docs = collection.get(ids=doc_nan_ids, include=['documents', 'metadatas'])
            nan_docs_df = pd.DataFrame({'nan_doc':nan_docs['documents'], 'source':nan_docs['ids']})
            nan_docs_df['source'] = nan_docs_df['source'].str.split('_').str[0]

            # 不足doc情報をマージ
            rrf_result_df = pd.merge(rrf_result_df , nan_docs_df, on='source', how='left')
            rrf_result_df['doc'] = rrf_result_df['doc'].fillna(rrf_result_df['nan_doc'])
            rrf_result_df = rrf_result_df.drop(columns=['nan_doc'])

        # Cohereでdocumentsをリランク(Rerank) =============
        results = COHERE_CLIENT.rerank(
            model=COHERE_RERANK_MODEL,
            query=query,
            documents=rrf_result_df['doc'].values.tolist(),
            top_n=4*k,
        ).results

        reranked_df = pd.DataFrame({
             'doc':    [rrf_result_df.loc[r.index, 'doc'   ] for r in results],
             'source': [rrf_result_df.loc[r.index, 'source'] for r in results],
             'score':  [r.relevance_score                    for r in results]  # rerank score
         })
        reranked_df['section'] = section
        # 各クエリの結果を保存
        conc_reranked_df = pd.concat([conc_reranked_df, reranked_df], axis=0, ignore_index=True) # 連結

        if trace:
            print(f"\n## {section}: retrieved distance, len={len(retrieved_df)}")
            display(retrieved_df)
            print(f"\n## {section}: BM25 Search Score, len={len(bm25score_df)}")
            display(bm25score_df)
            print(f"\n## {section}: rrf Score result, len={len(rrf_result_df)}")
            display(rrf_result_df)
            if doc_nan_ids != []:
                print(f"\n## {section}: nan_docs_df, len={len(nan_docs_df)}")
                display(nan_docs_df)
            print(f"\n## {section}: reranked score, len={len(reranked_df)}")
            display(reranked_df)

    # section毎のrerankを集約
    # スコア集約戦略: 相加平均 
    aggr_mean_reranked_df = (
        conc_reranked_df.groupby('source', as_index=False)['score'].sum()  # sourceで集約, スコアは合計
        .sort_values('score', ascending=False).reset_index(drop=True) # scoreでソート
    )
    aggr_mean_reranked_df['score'] = aggr_mean_reranked_df['score'] / n_queries # Queryの数で割り戻す

    # スコア集約戦略: 最大値
    aggr_max_reranked_df = (
        conc_reranked_df.groupby('source', as_index=False)['score'].max()  # sourceで集約, スコアは合計
        .sort_values('score', ascending=False).reset_index(drop=True) # scoreでソート
    )

    if aggr_strategy == 'MEAN' or aggr_strategy != 'MAX':
        aggr_reranked_df = aggr_mean_reranked_df
    else:
        aggr_reranked_df = aggr_max_reranked_df

    if trace:
        print(f"## concatenated and sorted reranked score")
        display(conc_reranked_df.sort_values(['score'], ascending=False))
        print(f"## mean aggregated reranked score; strategy=MEAN")
        display(aggr_mean_reranked_df)
        print(f"## max aggregated reranked score; strategy=MAX")
        display(aggr_max_reranked_df)

    return aggr_reranked_df[['source','score']].head(k).values.tolist()

### nDCG算出¶

#### nDCG用CSVデータ読み込み

In [12]:
# クエリ用CSVをDFに読み込み、クエリと関連度に分解
def read_ndcg_test_case(testcase_path='テストケース.csv'):
    # テストケース
    testcase_df = pd.read_csv(testcase_path, header=0)
    testcase_df[['purpose','materials','methods']] = testcase_df.groupby('test_id').ffill().fillna('')[['purpose','materials','methods']]
    # テスト用クエリ
    testquery_df = testcase_df[['test_id','purpose','materials','methods']].copy().drop_duplicates().reset_index(drop=True)

    for row, tid in enumerate(testquery_df['test_id']):
        no, nid, c = tid.split('_')
        query = notes_dic['実験ノート '+nid]
        if c != '2':
            testquery_df.loc[row, 'purpose']   = query['目的'].strip()
        testquery_df.loc[row, 'materials'] = query['材料'].strip()
        testquery_df.loc[row, 'methods']   = query['手順'].strip()

#    display(testquery_df)
    # 関連度
    relevance_df = testcase_df[['test_id','doc_id','gain']].rename(columns={'doc_id': 'name'}).copy()
    relevance_df['name'] = relevance_df['name'].str.strip()
#    display(relevance_df)

    print(f"\n### nDCG用テストデータ読込")
    print(f"### テストID - 関連度(1~0) - Note, relevance_df: len={len(relevance_df)}")
#    display(relevance_df)

    return testquery_df, relevance_df

testquery_df, relevance_df = read_ndcg_test_case(testcase_path='テストケース.csv')

print(f"## testquery_df")
display(testquery_df)
print(f"## relevance_df")
display(relevance_df)


### nDCG用テストデータ読込
### テストID - 関連度(1~0) - Note, relevance_df: len=125
## testquery_df


,test_id,purpose,materials,methods
0,1_ID1-2_1,1-1基準条件からNaOH量を半分に減らし、pH低下による感度変化や反応性低下の有無を評価し...,"精製水 20 mL\nHbA1c捕捉抗体1 (マウスモノクローナル,全長IgG) 1 mL\...",### 手順1\n\n1．試薬Aを作成\n1-1.①～③の分量を測定＆記録して、ビーカーに入...
1,2_ID1-2_2,,"精製水 20 mL\nHbA1c捕捉抗体1 (マウスモノクローナル,全長IgG) 1 mL\...",### 手順1\n\n1．試薬Aを作成\n1-1.①～③の分量を測定＆記録して、ビーカーに入...
2,3_ID2-17_1,Tris濃度が高い条件ではイオン強度やpH緩衝能が過剰となり、ラテックス凝集やバックグラウン...,"精製水 25 mL\nHbA1c捕捉抗体A (マウスモノクローナル,全長IgG) 1 mL\...",### 手順1\n\n1.試薬Aの基準調製\n1-1.材料①〜③の投入\n・①～③を順に秤量...
3,4_ID2-17_2,,"精製水 25 mL\nHbA1c捕捉抗体A (マウスモノクローナル,全長IgG) 1 mL\...",### 手順1\n\n1.試薬Aの基準調製\n1-1.材料①〜③の投入\n・①～③を順に秤量...
4,5_ID3-30_1,HEPES条件でラテックス粒径を非対称ペアとしタンパク質をBSAへ変更した構成で、感度・直線...,精製水 30 g\nHEPES 1.8 g\nIgGコートラテックス (粒径 110 nm)...,### 手順1\n\n1．試薬Aを作成\n1-1.①～④の分量を測定＆記録して、ビーカーに入...
5,6_ID3-30_2,,精製水 30 g\nHEPES 1.8 g\nIgGコートラテックス (粒径 110 nm)...,### 手順1\n\n1．試薬Aを作成\n1-1.①～④の分量を測定＆記録して、ビーカーに入...
6,7_ID1-3_1,1-1基準条件からNaOH量を2倍に増量し、高pH条件での感度向上の有無と、バックグラウンド...,"精製水 20 mL\nHbA1c捕捉抗体1 (マウスモノクローナル,全長IgG) 1 mL\...",### 手順1\n\n1．試薬Aを作成\n1-1.①～③の分量を測定＆記録して、ビーカーに入...
7,8_ID1-3_2,,"精製水 20 mL\nHbA1c捕捉抗体1 (マウスモノクローナル,全長IgG) 1 mL\...",### 手順1\n\n1．試薬Aを作成\n1-1.①～③の分量を測定＆記録して、ビーカーに入...
8,9_ID4-50_1,4-21〜4-22 および 4-41〜4-49 の検討結果を踏まえ、NPG/C8–C10 ポ...,原料1:ネオペンチルグリコール (NPG) 1.0 mol\n原料2:高純度C8–C10ミッ...,### 合成手順①\n\n1 L ジャケット付きガラス反応器に原料1〜3を秤量投入し、窒素 ...
9,10_ID4-50_2,,原料1:ネオペンチルグリコール (NPG) 1.0 mol\n原料2:高純度C8–C10ミッ...,### 合成手順①\n\n1 L ジャケット付きガラス反応器に原料1〜3を秤量投入し、窒素 ...


## relevance_df


,test_id,name,gain
0,1_ID1-2_1,実験ノート ID1-2,1.000000
1,1_ID1-2_1,実験ノート ID2-16,0.966667
2,1_ID1-2_1,実験ノート ID1-3,0.866667
3,1_ID1-2_1,実験ノート ID2-21,0.833333
4,1_ID1-2_1,実験ノート ID2-47,0.800000
5,1_ID1-2_1,実験ノート ID2-50,0.800000
6,1_ID1-2_1,実験ノート ID2-46,0.766667
7,1_ID1-2_1,実験ノート ID1-4,0.733333
8,1_ID1-2_1,実験ノート ID2-23,0.733333
9,1_ID1-2_1,実験ノート ID2-48,0.733333


#### nDCGスコア算出関数

In [13]:
def calc_ndcg(retrieved_df, relevance_df, k, trace=False): # -> ndcg
    # 読み込んだCSVクエリ-想定順位より、レシピデータセットを検索、
    # 検索結果のランクを出力、relevanceとマージしてnDCGスコアを計算
    # TopKのK値で足切りされた料理の順位ランクは0(同率最下位)とする
    ndcgs = []
    name_df = pd.DataFrame({'name': notes_dic.keys()}) # 料理名一覧
    # nCDG@K計算用に各データをマージ
    result_df = name_df.copy() # 料理名一覧
    result_df = pd.merge(result_df, retrieved_df, how='left')                  # 検索順位(TopKまで)をマージ
    result_df = pd.merge(result_df, relevance_df[['name','gain']], how='left') # 関連度をマージ
    result_df = result_df.dropna(subset=["score", "gain"], how="all")          # 両方NaNを削除
    result_df = result_df.fillna(0).sort_values('score', ascending=False).reset_index(drop=True).copy() # 残りを0埋め、ソート

    # nCDG@K (K=TopK)
    ndcg = ndcg_score(result_df['gain' ].to_numpy().reshape(1, -1), 
                      result_df['score'].to_numpy().reshape(1, -1),
                      k=k)
    if trace:
        print(f"## k={TopK}, nDCG Score = {ndcg:.6f}")
        display(result_df)

    return ndcg

#### Hybrid_without_user.dic

sudachi_user.dicを**空にしてから実行**のこと

In [17]:
trace = False

TopK = 10
MATE_NORMALIZE = False # 材料名寄せフラグ
aggr_strategy='MEAN'   # スコア集約戦略
hybrid_weight=[50, 50] # RRFでのVector/BM25検索のウェイト、デフォルト[50, 50]

ndcgs_df = pd.DataFrame({'Test_ID': testquery_df['test_id'].values.tolist() + ['#MEAN']}) # ndcgsの入れ物
name_df = pd.DataFrame({'name': notes_dic.keys()}) # ノート名一覧
for emb_type in list(EmbeddingType):       # モデル毎に
    EMBEDDING_MODEL = emb_type   # モデルを指定
    emb_model = str(EMBEDDING_MODEL).split('.')[1]

    print(f"\n### Case: Hybrid_Wo_User.dic,", 
          f"\nTopK={TopK}, embeddig={EMBEDDING_MODEL}, hybrid_weight={hybrid_weight},",
          f"MATE_NORMALIZE={MATE_NORMALIZE}, aggr_strategy={aggr_strategy}")

    collection, bm25index_dic, doc_ids = setup_stores(notes_dic)  # 次元が変わるため、collection(VectorStore)を再作成
    
    ndcgs = []
    test_ids = []
    for i, row in list(testquery_df.iterrows()):   # テストケース毎に
        queries = [row['purpose'].strip(), row['materials'].strip(), row['methods'].strip()]  # クエリを抽出
        queries_lst = ['\n'.join([s for s in queries if s])] + queries                        # allを作成
        queries_dic = {s: q if q != '' else '' for q, s in zip(queries_lst, ['all','目的','材料','手順'])}
        test_id = row['test_id']
        test_ids.append(test_id)
    
        # 検索実行（個別のクエリで検索）==============
        result = hybrid_retrieve(
            queries, collection, bm25index_dic, doc_ids, 
            k=TopK, hybrid_weight=hybrid_weight, aggr_strategy=aggr_strategy, trace=trace
        )
        retrieved_df = pd.DataFrame({
            'name': [ r[0] for r in result], 
            'model': emb_model,
            'score': [ r[1] for r in result]
        })
        retrieved_df = retrieved_df.sort_values('score', ascending=False)
        print(f"\n## {i+1}, Retrieval Result: test_id={test_id}")
        display(retrieved_df)

        ndcg = calc_ndcg(retrieved_df, relevance_df.query("test_id==@test_id"), k=TopK, trace=True)
        ndcgs.append(ndcg)
        print(f"## {test_id}: nDCG Score = {ndcg:.6f}")

        time.sleep(30)  # リクエストレート制限対策
    
    ndcgs_mean = np.array(ndcgs).mean()
    print(f"\n### Case: Hybrid_Wo_User.dic, EMBEDDING_MODEL={EMBEDDING_MODEL}: nDCG Score Mean = {ndcgs_mean:.6f}")
    wk_df = pd.DataFrame({'Test_ID':test_ids + ['#MEAN'], f"{emb_model} nDCG Score":ndcgs + [ndcgs_mean]})
    ndcgs_df = pd.merge(ndcgs_df, wk_df, how='left')
    display(ndcgs_df)

hybrid_wo_userdic_df = ndcgs_df.assign(logic='Hybrid_wo_User.dic')
display(hybrid_wo_userdic_df)


### Case: Hybrid_Wo_User.dic, 
TopK=10, embeddig=EmbeddingType.GEMINI, hybrid_weight=[50, 50], MATE_NORMALIZE=False, aggr_strategy=MEAN


100%|███████████████████████████████████████████| 34/34 [01:07<00:00,  1.99s/it]



## Vector Store、BM25 Indexに追加された文書数: 1360

## 1, Retrieval Result: test_id=1_ID1-2_1


,name,model,score
0,実験ノート ID1-6,GEMINI,0.771767
1,実験ノート ID1-1,GEMINI,0.750386
2,実験ノート ID1-2,GEMINI,0.718045
3,実験ノート ID2-16,GEMINI,0.637596
4,実験ノート ID1-5,GEMINI,0.612432
5,実験ノート ID1-3,GEMINI,0.569345
6,実験ノート ID1-10,GEMINI,0.537368
7,実験ノート ID1-48,GEMINI,0.530670
8,実験ノート ID1-11,GEMINI,0.528688
9,実験ノート ID2-9,GEMINI,0.520083


,name,model,score,gain
0,実験ノート ID1-6,GEMINI,0.771767,0.000000
1,実験ノート ID1-1,GEMINI,0.750386,0.733333
2,実験ノート ID1-2,GEMINI,0.718045,1.000000
3,実験ノート ID2-16,GEMINI,0.637596,0.966667
4,実験ノート ID1-5,GEMINI,0.612432,0.000000
5,実験ノート ID1-3,GEMINI,0.569345,0.866667
6,実験ノート ID1-10,GEMINI,0.537368,0.000000
7,実験ノート ID1-48,GEMINI,0.530670,0.000000
8,実験ノート ID1-11,GEMINI,0.528688,0.000000
9,実験ノート ID2-9,GEMINI,0.520083,0.000000


...

,name,model,score
0,実験ノート ID4-50,RURI,0.942902
1,実験ノート ID4-45,RURI,0.912919
2,実験ノート ID4-44,RURI,0.887966
3,実験ノート ID4-43,RURI,0.875553
4,実験ノート ID4-22,RURI,0.852885
5,実験ノート ID4-41,RURI,0.849626
6,実験ノート ID4-9,RURI,0.839185
7,実験ノート ID4-47,RURI,0.831403
8,実験ノート ID4-26,RURI,0.817085
9,実験ノート ID4-28,RURI,0.810572


,name,model,score,gain
0,実験ノート ID4-50,RURI,0.942902,1.00
1,実験ノート ID4-45,RURI,0.912919,0.80
2,実験ノート ID4-44,RURI,0.887966,0.75
3,実験ノート ID4-43,RURI,0.875553,0.70
4,実験ノート ID4-22,RURI,0.852885,0.70
5,実験ノート ID4-41,RURI,0.849626,0.70
6,実験ノート ID4-9,RURI,0.839185,0.65
7,実験ノート ID4-47,RURI,0.831403,0.00
8,実験ノート ID4-26,RURI,0.817085,0.65
9,実験ノート ID4-28,RURI,0.810572,0.00


## k=10, nDCG Score = 0.883742
## 10_ID4-50_2: nDCG Score = 0.883742

### Case: Hybrid_Wo_User.dic, EMBEDDING_MODEL=EmbeddingType.RURI: nDCG Score Mean = 0.645701


,Test_ID,GEMINI nDCG Score,OPENAI_SMALL nDCG Score,OPENAI_LARGE nDCG Score,PLAMO nDCG Score,RURI nDCG Score
0,1_ID1-2_1,0.430956,0.404317,0.453925,0.487322,0.388753
1,2_ID1-2_2,0.576626,0.609217,0.570914,0.589077,0.582838
2,3_ID2-17_1,0.707362,0.707362,0.707362,0.707362,0.707362
3,4_ID2-17_2,0.779554,0.779554,0.779554,0.779554,0.779554
4,5_ID3-30_1,0.632675,0.632675,0.690839,0.690839,0.563278
5,6_ID3-30_2,0.602577,0.608695,0.599694,0.602577,0.531990
6,7_ID1-3_1,0.541538,0.399136,0.565332,0.446534,0.467640
7,8_ID1-3_2,0.572782,0.623161,0.584651,0.530873,0.612531
8,9_ID4-50_1,0.939319,0.939319,0.939319,0.939319,0.939319
9,10_ID4-50_2,0.883742,0.883742,0.883742,0.883742,0.883742


,Test_ID,GEMINI nDCG Score,OPENAI_SMALL nDCG Score,OPENAI_LARGE nDCG Score,PLAMO nDCG Score,RURI nDCG Score,logic
0,1_ID1-2_1,0.430956,0.404317,0.453925,0.487322,0.388753,Hybrid_Wo_User.dic
1,2_ID1-2_2,0.576626,0.609217,0.570914,0.589077,0.582838,Hybrid_Wo_User.dic
2,3_ID2-17_1,0.707362,0.707362,0.707362,0.707362,0.707362,Hybrid_Wo_User.dic
3,4_ID2-17_2,0.779554,0.779554,0.779554,0.779554,0.779554,Hybrid_Wo_User.dic
4,5_ID3-30_1,0.632675,0.632675,0.690839,0.690839,0.563278,Hybrid_Wo_User.dic
5,6_ID3-30_2,0.602577,0.608695,0.599694,0.602577,0.531990,Hybrid_Wo_User.dic
6,7_ID1-3_1,0.541538,0.399136,0.565332,0.446534,0.467640,Hybrid_Wo_User.dic
7,8_ID1-3_2,0.572782,0.623161,0.584651,0.530873,0.612531,Hybrid_Wo_User.dic
8,9_ID4-50_1,0.939319,0.939319,0.939319,0.939319,0.939319,Hybrid_Wo_User.dic
9,10_ID4-50_2,0.883742,0.883742,0.883742,0.883742,0.883742,Hybrid_Wo_User.dic


### nDCGまとめ

In [18]:
display(hybrid_wo_userdic_df)

,Test_ID,GEMINI nDCG Score,OPENAI_SMALL nDCG Score,OPENAI_LARGE nDCG Score,PLAMO nDCG Score,RURI nDCG Score,logic
0,1_ID1-2_1,0.430956,0.404317,0.453925,0.487322,0.388753,Hybrid_Wo_User.dic
1,2_ID1-2_2,0.576626,0.609217,0.570914,0.589077,0.582838,Hybrid_Wo_User.dic
2,3_ID2-17_1,0.707362,0.707362,0.707362,0.707362,0.707362,Hybrid_Wo_User.dic
3,4_ID2-17_2,0.779554,0.779554,0.779554,0.779554,0.779554,Hybrid_Wo_User.dic
4,5_ID3-30_1,0.632675,0.632675,0.690839,0.690839,0.563278,Hybrid_Wo_User.dic
5,6_ID3-30_2,0.602577,0.608695,0.599694,0.602577,0.531990,Hybrid_Wo_User.dic
6,7_ID1-3_1,0.541538,0.399136,0.565332,0.446534,0.467640,Hybrid_Wo_User.dic
7,8_ID1-3_2,0.572782,0.623161,0.584651,0.530873,0.612531,Hybrid_Wo_User.dic
8,9_ID4-50_1,0.939319,0.939319,0.939319,0.939319,0.939319,Hybrid_Wo_User.dic
9,10_ID4-50_2,0.883742,0.883742,0.883742,0.883742,0.883742,Hybrid_Wo_User.dic


#### 縦持ちに変換

In [20]:
def convert_to_long(in_df):
    out_df = in_df.melt(
        id_vars=['Test_ID', 'logic'],   # 固定で残す列
        var_name='emb_model',           # 元の列名が入る列名
        value_name='nDCG Score'         # 値の列名
    )
    return out_df

long1_df = convert_to_long(hybrid_wo_userdic_df)

nDCG_long_df = pd.concat([long1_df], axis=0, ignore_index=True)
nDCG_long_df['emb_model'] = nDCG_long_df['emb_model'].str.replace(' nDCG Score', '', regex=False) # 列名をモデル名にする

display(nDCG_long_df)

# CSVに書き出し
nDCG_long_df.to_csv('Note_nDCG2_long_df.csv', header=True, index=False) # CSV保存

def display_maxrows(df): # 表示行数制限を一時的に回避
    bk_pd_max_rows = pd.options.display.max_rows
    pd.options.display.max_rows = None
    display(df)    
    pd.options.display.max_rows = bk_pd_max_rows
display_maxrows(nDCG_long_df)

,Test_ID,logic,emb_model,nDCG Score
0,1_ID1-2_1,Hybrid_Wo_User.dic,GEMINI,0.430956
1,2_ID1-2_2,Hybrid_Wo_User.dic,GEMINI,0.576626
2,3_ID2-17_1,Hybrid_Wo_User.dic,GEMINI,0.707362
3,4_ID2-17_2,Hybrid_Wo_User.dic,GEMINI,0.779554
4,5_ID3-30_1,Hybrid_Wo_User.dic,GEMINI,0.632675
5,6_ID3-30_2,Hybrid_Wo_User.dic,GEMINI,0.602577
6,7_ID1-3_1,Hybrid_Wo_User.dic,GEMINI,0.541538
7,8_ID1-3_2,Hybrid_Wo_User.dic,GEMINI,0.572782
8,9_ID4-50_1,Hybrid_Wo_User.dic,GEMINI,0.939319
9,10_ID4-50_2,Hybrid_Wo_User.dic,GEMINI,0.883742


,Test_ID,logic,emb_model,nDCG Score
0,1_ID1-2_1,Hybrid_Wo_User.dic,GEMINI,0.430956
1,2_ID1-2_2,Hybrid_Wo_User.dic,GEMINI,0.576626
2,3_ID2-17_1,Hybrid_Wo_User.dic,GEMINI,0.707362
3,4_ID2-17_2,Hybrid_Wo_User.dic,GEMINI,0.779554
4,5_ID3-30_1,Hybrid_Wo_User.dic,GEMINI,0.632675
5,6_ID3-30_2,Hybrid_Wo_User.dic,GEMINI,0.602577
6,7_ID1-3_1,Hybrid_Wo_User.dic,GEMINI,0.541538
7,8_ID1-3_2,Hybrid_Wo_User.dic,GEMINI,0.572782
8,9_ID4-50_1,Hybrid_Wo_User.dic,GEMINI,0.939319
9,10_ID4-50_2,Hybrid_Wo_User.dic,GEMINI,0.883742
